In [1]:
import pandas as pd
import numpy as np
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_df

In [4]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\background_files\jerusalem_muni'.format(education_folder_path),'מוסדות בירושלים 2020')
JLM=remove_spaces_in_columns(JLM)

In [5]:
#קאורדינטות של מוסדות חינוך בכל הארץ ממשרד החינוך
JTMT_setls_schools_coordinates_with_src=up_load_df(r'{}\background_files'.format(education_folder_path),'JTMT_setls_schools_coordinates_with_src')
JTMT_setls_schools_coordinates_with_src=remove_spaces_in_columns(JTMT_setls_schools_coordinates_with_src)

In [6]:
JLM.rename(columns={'סמל_חינוך': 'סמל_מוסד'}, inplace=True)

JLM = JLM.drop(columns=['פיקוח', 'מעמד_משפטי', "מס'_תלמידים_ז-ט", "מס'_תלמידים_י-יד", 'קוד_אזור_סטיסטי', 'תאור_אזור_סטטיסטי', 'Unnamed:_18', 'Unnamed:_19'])

JLM=JLM[JLM['שלב_חינוך'] != 'גני ילדים']

In [7]:
JTMT_setls_schools_coordinates_with_src.head()

,סמל_מוסד,coordinate_x,coordinate_y,SRC
0,110023,219040.00,633692.00,moe_mosdot_coordinates
1,110098,190867.00,634787.00,moe_mosdot_coordinates
2,110130,220523.00,630416.00,moe_mosdot_coordinates
3,110155,218115.00,632094.00,moe_mosdot_coordinates
4,110171,220854.00,629272.00,moe_mosdot_coordinates


In [8]:
rows_with_zero = JLM[JLM['coordinate_x'] == 0]
JLM = JLM[JLM['coordinate_x'] != 0]

In [ ]:
JLM['SRC'] = 'jerusalem_muni'

,סמל_עירייה,סמל_מוסד,שם_מוסד,אגף,שלב_חינוך,סוג_חינוך,מכיתה,עד_כיתה,מספר_כיתות,"סה""כ_תלמידים",coordinate_x,coordinate_y,SRC
0,21321.00,110106.00,"מ""מ אגרון",כללי,יסודי,רגיל,א,ו,8.00,230.00,216876.00,629987.00,jerusalem_muni
1,21323.00,110312.00,"מ""מ ארגנטינה הניסויי",כללי,יסודי,רגיל,א,ח,12.00,347.00,216154.00,630603.00,jerusalem_muni
2,21326.00,110155.00,"מ""מ בית הכרם -הישראלי",כללי,יסודי,רגיל,א,ו,17.00,534.00,218166.00,632109.00,jerusalem_muni
3,21327.00,110171.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",כללי,יסודי,רגיל,א,ו,13.00,331.00,220805.00,629266.00,jerusalem_muni
4,21328.00,111112.00,"מ""מ זלמן ארן",כללי,יסודי,רגיל,א,ו,13.00,367.00,220633.00,628408.00,jerusalem_muni


In [10]:
JLM['SRC'] = 'jerusalem_muni'
rows_with_zero['SRC'] = ''

In [11]:
# מיזוג הטבלאות לפי העמודה "סמל_מוסד"
rows_with_zero = pd.merge(rows_with_zero, JTMT_setls_schools_coordinates_with_src[['סמל_מוסד', 'coordinate_x', 'coordinate_y', 'SRC']],
                     on='סמל_מוסד', how='left')

# עדכון העמודות המקוריות
rows_with_zero['coordinate_x'] = rows_with_zero['coordinate_x_y']
rows_with_zero['coordinate_y'] = rows_with_zero['coordinate_y_y']
rows_with_zero['SRC'] = rows_with_zero['SRC_y']

# מחיקת העמודות המיותרות
rows_with_zero = rows_with_zero.drop(['coordinate_x_x', 'coordinate_y_x', 'coordinate_x_y', 'coordinate_y_y', 'SRC_x', 'SRC_y'], axis=1)
rows_with_zero.head()

,סמל_עירייה,סמל_מוסד,שם_מוסד,אגף,שלב_חינוך,סוג_חינוך,מכיתה,עד_כיתה,מספר_כיתות,"סה""כ_תלמידים",coordinate_x,coordinate_y,SRC
0,23425.00,110932.00,בית וגן - בנים (לויכטר),חרדי,יסודי,רגיל,א,ח,9.00,278.00,217291.00,630454.00,moe_mosdot_coordinates
1,31475.00,358127.00,"מ""מ מגשימים",כללי,רב גילאי,מיוחד,ז,יב,7.00,53.00,219533.00,629760.00,moe_mosdot_coordinates
2,51504.00,141010.00,"חט""ב מכללת אורט גבעת רם",כללי,על יסודי,רגיל,ז,ט,6.00,178.00,218594.00,631431.00,moe_mosdot_coordinates
3,53100.00,199109.00,אפיקי דעה,חרדי,על יסודי,רגיל,ט,יד,6.00,147.00,218088.00,636736.00,moe_mosdot_coordinates
4,53122.00,343996.00,נתיב החינוך,חרדי,על יסודי,רגיל,ט,יד,8.00,229.00,218088.00,636736.00,moe_mosdot_coordinates


In [12]:
# מיזוג הטבלאות
JLM=pd.concat([JLM, rows_with_zero], ignore_index=True)

In [14]:
JLM['סמל_מוסד'] = JLM['סמל_עירייה']
JLM.drop(columns='סמל_עירייה', inplace=True)

In [15]:
is_unique = JLM['סמל_מוסד'].is_unique
is_unique

True

In [16]:
JLM.head()

,סמל_מוסד,שם_מוסד,אגף,שלב_חינוך,סוג_חינוך,מכיתה,עד_כיתה,מספר_כיתות,"סה""כ_תלמידים",coordinate_x,coordinate_y,SRC
0,21321.00,"מ""מ אגרון",כללי,יסודי,רגיל,א,ו,8.00,230.00,216876.00,629987.00,jerusalem_muni
1,21323.00,"מ""מ ארגנטינה הניסויי",כללי,יסודי,רגיל,א,ח,12.00,347.00,216154.00,630603.00,jerusalem_muni
2,21326.00,"מ""מ בית הכרם -הישראלי",כללי,יסודי,רגיל,א,ו,17.00,534.00,218166.00,632109.00,jerusalem_muni
3,21327.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",כללי,יסודי,רגיל,א,ו,13.00,331.00,220805.00,629266.00,jerusalem_muni
4,21328.00,"מ""מ זלמן ארן",כללי,יסודי,רגיל,א,ו,13.00,367.00,220633.00,628408.00,jerusalem_muni


In [17]:
# הגדרת רשימת הכיתות לטווח א' עד ח'
class_map = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד']

# הגדרת עמודות לכיתות עם NaN כערך ברירת מחדל
for single_class in class_map:
    JLM[f'כיתה_{single_class}'] = np.nan

# פונקציה לחישוב מספר התלמידים לכל כיתה בטווח
def first_15_division(row):
    start_idx = class_map.index(row['מכיתה'])
    end_idx = class_map.index(row['עד_כיתה'])
    class_range = class_map[start_idx:end_idx + 1]

    students_remaining = row['סה"כ_תלמידים']
    for single_class in class_range:
        students_in_class = min(15, students_remaining)
        row[f'כיתה_{single_class}'] = students_in_class
        students_remaining -= students_in_class

    row['סה"כ_תלמידים'] = students_remaining
    return row


# החלת הפונקציה על כל השורות בטבלה
JLM = JLM.apply(first_15_division, axis=1)

# הצגת הטבלה החדשה
JLM.head()

,סמל_מוסד,שם_מוסד,אגף,שלב_חינוך,סוג_חינוך,מכיתה,עד_כיתה,מספר_כיתות,"סה""כ_תלמידים",coordinate_x,coordinate_y,SRC,כיתה_א,כיתה_ב,כיתה_ג,כיתה_ד,כיתה_ה,כיתה_ו,כיתה_ז,כיתה_ח,כיתה_ט,כיתה_י,כיתה_יא,כיתה_יב,כיתה_יג,כיתה_יד
0,21321.00,"מ""מ אגרון",כללי,יסודי,רגיל,א,ו,8.00,140.00,216876.00,629987.00,jerusalem_muni,15.00,15.00,15.00,15.00,15.00,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21323.00,"מ""מ ארגנטינה הניסויי",כללי,יסודי,רגיל,א,ח,12.00,227.00,216154.00,630603.00,jerusalem_muni,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,NaN,NaN,NaN,NaN,NaN,NaN
2,21326.00,"מ""מ בית הכרם -הישראלי",כללי,יסודי,רגיל,א,ו,17.00,444.00,218166.00,632109.00,jerusalem_muni,15.00,15.00,15.00,15.00,15.00,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21327.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",כללי,יסודי,רגיל,א,ו,13.00,241.00,220805.00,629266.00,jerusalem_muni,15.00,15.00,15.00,15.00,15.00,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21328.00,"מ""מ זלמן ארן",כללי,יסודי,רגיל,א,ו,13.00,277.00,220633.00,628408.00,jerusalem_muni,15.00,15.00,15.00,15.00,15.00,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# פונקציה לחלוקת התלמידים הנותרים שווה בשווה
def equal_distribution_students(row):
    # איתור הטווח של הכיתות
    start_idx = class_map.index(row['מכיתה'])
    end_idx = class_map.index(row['עד_כיתה'])
    class_range = class_map[start_idx:end_idx + 1]
    
    # חישוב מספר התלמידים הנותרים
    students_remaining = row['סה"כ_תלמידים']
    num_of_classes = len(class_range)
    
    if num_of_classes > 0:
        # חלוקת התלמידים הנותרים שווה בשווה לכיתות בטווח
        students_per_class = students_remaining / num_of_classes
        for single_class in class_range:
            # אם יש כבר ערך בעמודת הכיתה, נוסיף את הערך החדש
            if pd.notna(row[f'כיתה_{single_class}']):
                row[f'כיתה_{single_class}'] += students_per_class
            else:
                row[f'כיתה_{single_class}'] = students_per_class

    # עדכון סה"כ תלמידים לאחר חלוקה
    row['סה"כ_תלמידים'] = 0
    return row

# החלת הפונקציה על כל השורות בטבלה
JLM = JLM.apply(equal_distribution_students, axis=1)

# הצגת הטבלה החדשה
JLM.head()

,סמל_מוסד,שם_מוסד,אגף,שלב_חינוך,סוג_חינוך,מכיתה,עד_כיתה,מספר_כיתות,"סה""כ_תלמידים",coordinate_x,coordinate_y,SRC,כיתה_א,כיתה_ב,כיתה_ג,כיתה_ד,כיתה_ה,כיתה_ו,כיתה_ז,כיתה_ח,כיתה_ט,כיתה_י,כיתה_יא,כיתה_יב,כיתה_יג,כיתה_יד
0,21321.00,"מ""מ אגרון",כללי,יסודי,רגיל,א,ו,8.00,0,216876.00,629987.00,jerusalem_muni,38.33,38.33,38.33,38.33,38.33,38.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21323.00,"מ""מ ארגנטינה הניסויי",כללי,יסודי,רגיל,א,ח,12.00,0,216154.00,630603.00,jerusalem_muni,43.38,43.38,43.38,43.38,43.38,43.38,43.38,43.38,NaN,NaN,NaN,NaN,NaN,NaN
2,21326.00,"מ""מ בית הכרם -הישראלי",כללי,יסודי,רגיל,א,ו,17.00,0,218166.00,632109.00,jerusalem_muni,89.00,89.00,89.00,89.00,89.00,89.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21327.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",כללי,יסודי,רגיל,א,ו,13.00,0,220805.00,629266.00,jerusalem_muni,55.17,55.17,55.17,55.17,55.17,55.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21328.00,"מ""מ זלמן ארן",כללי,יסודי,רגיל,א,ו,13.00,0,220633.00,628408.00,jerusalem_muni,61.17,61.17,61.17,61.17,61.17,61.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
class_map = ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד']

# בדיקה ועדכון כל תא אם מספר התלמידים קטן מ-15
for single_class in class_map:
    JLM[single_class] = JLM[single_class].apply(lambda x: x if pd.isna(x) or x >= 15 else 0)

# הצגת הטבלה המעודכנת
JLM.head()

,סמל_מוסד,שם_מוסד,אגף,שלב_חינוך,סוג_חינוך,מכיתה,עד_כיתה,מספר_כיתות,"סה""כ_תלמידים",coordinate_x,coordinate_y,SRC,כיתה_א,כיתה_ב,כיתה_ג,כיתה_ד,כיתה_ה,כיתה_ו,כיתה_ז,כיתה_ח,כיתה_ט,כיתה_י,כיתה_יא,כיתה_יב,כיתה_יג,כיתה_יד
0,21321.00,"מ""מ אגרון",כללי,יסודי,רגיל,א,ו,8.00,0,216876.00,629987.00,jerusalem_muni,38.33,38.33,38.33,38.33,38.33,38.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21323.00,"מ""מ ארגנטינה הניסויי",כללי,יסודי,רגיל,א,ח,12.00,0,216154.00,630603.00,jerusalem_muni,43.38,43.38,43.38,43.38,43.38,43.38,43.38,43.38,NaN,NaN,NaN,NaN,NaN,NaN
2,21326.00,"מ""מ בית הכרם -הישראלי",כללי,יסודי,רגיל,א,ו,17.00,0,218166.00,632109.00,jerusalem_muni,89.00,89.00,89.00,89.00,89.00,89.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21327.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",כללי,יסודי,רגיל,א,ו,13.00,0,220805.00,629266.00,jerusalem_muni,55.17,55.17,55.17,55.17,55.17,55.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21328.00,"מ""מ זלמן ארן",כללי,יסודי,רגיל,א,ו,13.00,0,220633.00,628408.00,jerusalem_muni,61.17,61.17,61.17,61.17,61.17,61.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# חלוקה לשכבות גיל
ages = {
    'ele': ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו'],
    'mid': ['כיתה_ז', 'כיתה_ח', 'כיתה_ט'],
    'high': ['כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד']
}

# חישוב מספר התלמידים לכל שכבה
JLM['ele_stu'] = JLM[ages['ele']].sum(axis=1)
JLM['mid_stu'] = JLM[ages['mid']].sum(axis=1)
JLM['high_stu'] = JLM[ages['high']].sum(axis=1)

#sector
JLM.loc[JLM['אגף']=='ערבי','sector'] = 1
JLM.loc[JLM['אגף']=='חרדי','sector'] = 2
JLM.loc[JLM['אגף']=='כללי','sector'] = 3

JLM_final = JLM.drop(['אגף', 'שלב_חינוך', 'כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד', 'מספר_כיתות', 'סה"כ_תלמידים', 'מכיתה', 'עד_כיתה', 'שם_מוסד', 'סוג_חינוך'], axis=1)

JLM_final.rename(columns={'סמל_מוסד': 'ID'}, inplace=True)

JLM_final.rename(columns={'coordinate_x': 'x'}, inplace=True)

JLM_final.rename(columns={'coordinate_y': 'y'}, inplace=True)

In [21]:
JLM.head()

,סמל_מוסד,שם_מוסד,אגף,שלב_חינוך,סוג_חינוך,מכיתה,עד_כיתה,מספר_כיתות,"סה""כ_תלמידים",coordinate_x,coordinate_y,SRC,כיתה_א,כיתה_ב,כיתה_ג,כיתה_ד,כיתה_ה,כיתה_ו,כיתה_ז,כיתה_ח,כיתה_ט,כיתה_י,כיתה_יא,כיתה_יב,כיתה_יג,כיתה_יד,ele_stu,mid_stu,high_stu,sector
0,21321.00,"מ""מ אגרון",כללי,יסודי,רגיל,א,ו,8.00,0,216876.00,629987.00,jerusalem_muni,38.33,38.33,38.33,38.33,38.33,38.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230.00,0.00,0.00,3.00
1,21323.00,"מ""מ ארגנטינה הניסויי",כללי,יסודי,רגיל,א,ח,12.00,0,216154.00,630603.00,jerusalem_muni,43.38,43.38,43.38,43.38,43.38,43.38,43.38,43.38,NaN,NaN,NaN,NaN,NaN,NaN,260.25,86.75,0.00,3.00
2,21326.00,"מ""מ בית הכרם -הישראלי",כללי,יסודי,רגיל,א,ו,17.00,0,218166.00,632109.00,jerusalem_muni,89.00,89.00,89.00,89.00,89.00,89.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,534.00,0.00,0.00,3.00
3,21327.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",כללי,יסודי,רגיל,א,ו,13.00,0,220805.00,629266.00,jerusalem_muni,55.17,55.17,55.17,55.17,55.17,55.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,331.00,0.00,0.00,3.00
4,21328.00,"מ""מ זלמן ארן",כללי,יסודי,רגיל,א,ו,13.00,0,220633.00,628408.00,jerusalem_muni,61.17,61.17,61.17,61.17,61.17,61.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,367.00,0.00,0.00,3.00


In [22]:
JLM_final.head()

,ID,x,y,SRC,ele_stu,mid_stu,high_stu,sector
0,21321.00,216876.00,629987.00,jerusalem_muni,230.00,0.00,0.00,3.00
1,21323.00,216154.00,630603.00,jerusalem_muni,260.25,86.75,0.00,3.00
2,21326.00,218166.00,632109.00,jerusalem_muni,534.00,0.00,0.00,3.00
3,21327.00,220805.00,629266.00,jerusalem_muni,331.00,0.00,0.00,3.00
4,21328.00,220633.00,628408.00,jerusalem_muni,367.00,0.00,0.00,3.00


In [23]:
JLM.to_excel('JLM_1.xlsx', index=False)
JLM_final.to_excel('JLM_final_1.xlsx', index=False)